In [34]:
#import dependencies
import pandas as pd
import os

In [35]:
#set file_path for items_complete.csv
file_path_text = os.path.join("..", "generated_data","items_complete.csv")

#set file_path for players_complete.csv
file_path_players = os.path.join("..", "generated_data","players_complete.csv")

#set file_path for purchase_data_3.csv
file_path_purchase = os.path.join("..", "generated_data","purchase_data_3.csv")


In [36]:
#read in text
items = pd.read_csv(file_path_text)
items.head()

#read in players
players = pd.read_csv(file_path_players)
players.head()

#read in purhcase
purchase = pd.read_csv(file_path_purchase)
purchase.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,1,Aidaira26,21,Male,12,Dawne,3.36
2,2,Irim47,17,Male,5,Putrid Fan,2.63
3,3,Irith83,17,Male,123,Twilight's Carver,2.55
4,4,Philodil43,22,Male,154,Feral Katana,4.11


In [37]:
#Player Count
total_players = pd.DataFrame({"Total Players": players["SN"].count()}, index=[0] )
total_players

,Total Players
0,1163


In [38]:
#Purchasing Analysis (Total)

#Number of Unique Items
unique_items = items["Item Name"].nunique()
#Average Purchase Price
avg_purchase_price = purchase["Price"].mean()
#Total Number of Purchases
total_num_purchase = purchase["Purchase ID"].count()
#Total Revenue
total_revenue = purchase["Price"].sum()

purchasing_analysis = pd.DataFrame({"Number of Unique Items":unique_items,
                                   "Average Price":"${:.2f}".format(avg_purchase_price),
                                   "Number of Purchases":total_num_purchase,
                                   "Total Revenue":"${:.2f}".format(total_revenue)}, index=[0])

purchasing_analysis[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,186,$2.92,78,$228.10


In [39]:
#Gender Demographics

count_players = players["Gender"].value_counts()
total_players = players["Player ID"].count()
percentage = round((count_players / total_players)*100, 2).map("{:}%".format)


demographics = pd.DataFrame({"Percentage of Players": percentage, "Total Count": count_players})
demographics

,Percentage of Players,Total Count
Male,82.03%,954
Female,16.08%,187
Other / Non-Disclosed,1.89%,22


In [40]:
#Purchasing Analysis (Gender) - The below each broken by gender

grouped = purchase.groupby("Gender")

#Purchase Count
count_purchase = grouped["Purchase ID"].count()
#Average Purchase Price
average_purchase = grouped["Price"].mean()
#Total Purchase Value
total_purchase = count_purchase * average_purchase

#Create a dataframe
purchasing_analysis = pd.DataFrame({"Purchase Count":count_purchase,
                                   "Average Purchase Price":average_purchase,
                                   "Total Purchase Value":total_purchase,
                                   })


#---Calculate min-max normalization between the average purchase price for genders to map values from range 0 to 1---

#get max and min average purchase price between female, male, other 
max_purchase_price = purchasing_analysis["Average Purchase Price"].max()
min_purchase_price = purchasing_analysis["Average Purchase Price"].min()

#Calculate a min-max normalization on average purchase price
normalized_totals = (purchasing_analysis["Average Purchase Price"] - min_purchase_price) / (max_purchase_price - min_purchase_price)

#Add new column named Normalized Totals to df 
purchasing_analysis["Normalized Totals"] = normalized_totals

#format columns to include $ signs
purchasing_analysis["Average Purchase Price"] = purchasing_analysis["Average Purchase Price"].map("$ {:,.2f}".format)
purchasing_analysis["Total Purchase Value"] = purchasing_analysis["Total Purchase Value"].map("$ {:,.2f}".format)

#reorder columns
purchasing_analysis = purchasing_analysis[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
purchasing_analysis


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,13,$ 3.18,$ 41.38,1.000000
Male,64,$ 2.88,$ 184.60,0.719021
Other / Non-Disclosed,1,$ 2.12,$ 2.12,0.000000


In [41]:
#Age Demographics - The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

bins = [0,10,14,19,24,29,34,39,100]
group_names = ["<10", "10-14" , "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#cut players and place Age in bins and put it in new column called age group
players["Age Group"] = pd.cut(players["Age"],bins,labels=group_names)

#group by summary bin column
group = players.groupby("Age Group")

#get count of players per bin
age_per_bin = group["Player ID"].count()

#get the total count of players
total = age_per_bin.sum()

#calculate percentage of players per bin and format it with percentages
percentage_of_players = round(age_per_bin/total*100,2).map("{:}%".format)

age_demographics = pd.DataFrame({"Percentage of Players": percentage_of_players,
                                 "Total Count": age_per_bin})
age_demographics


,Percentage of Players,Total Count
Age Group,,
<10,5.33%,62
10-14,3.96%,46
15-19,17.54%,204
20-24,41.79%,486
25-29,16.25%,189
30-34,8.17%,95
35-39,5.16%,60
40+,1.81%,21


In [42]:
#Purchasing_Analysis (Age)

#cut purchase and place Age in bins and put it in new column called summary
purchase["Age Group"] = pd.cut(purchase["Age"],bins,labels=group_names)

grouped_purchase = purchase.groupby("Age Group")

#Purchase Count
count_purchase_age = grouped_purchase["Purchase ID"].count()
#Average Purchase Price
average_purchase_age = grouped_purchase["Price"].mean()
#Total Purchase Value
total_purchase_age = count_purchase_age * average_purchase_age


#create new dataframe to hold everything
purchasing_analysis_age = pd.DataFrame({"Purchase Count": count_purchase_age,
                                       "Average Purchase Price": average_purchase_age,
                                      "Total Purchase Value": total_purchase_age})




#---Calculate min-max normalization between the average purchase price for genders to map values from range 0 to 1---

#get max and min average purchase price between ages
max_purchase_price_age = purchasing_analysis_age["Average Purchase Price"].max()
min_purchase_price_age = purchasing_analysis_age["Average Purchase Price"].min()

#Calculate a min-max normalization on average purchase price
normalized_totals_age = (purchasing_analysis_age["Average Purchase Price"] - min_purchase_price_age) / (max_purchase_price_age - min_purchase_price_age)

#Add new column named Normalized Totals to df 
purchasing_analysis_age["Normalized Totals"] = normalized_totals_age

#format columns to include $ signs
purchasing_analysis_age["Average Purchase Price"] = purchasing_analysis_age["Average Purchase Price"].map("$ {:,.2f}".format)
purchasing_analysis_age["Total Purchase Value"] = purchasing_analysis_age["Total Purchase Value"].map("$ {:,.2f}".format)

#reorder columns
purchasing_analysis_age = purchasing_analysis_age[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
purchasing_analysis_age


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,5,$ 2.76,$ 13.82,0.292497
10-14,3,$ 2.99,$ 8.96,0.376027
15-19,11,$ 2.76,$ 30.41,0.292702
20-24,36,$ 3.02,$ 108.89,0.390303
25-29,9,$ 2.90,$ 26.11,0.343932
30-34,7,$ 1.98,$ 13.89,0.000000
35-39,6,$ 3.56,$ 21.37,0.591729
40+,1,$ 4.65,$ 4.65,1.000000


In [43]:
#Top Spenders

#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

#SN
grouped_sn = purchase.groupby("SN")

#Purchase Count
count_purchase_sn = grouped_sn["Purchase ID"].count()

#Average Purchase Price
average_purchase_sn = grouped_sn["Price"].mean()

#Total Purchase Value
total_purchase_sn = grouped_sn["Price"].sum()

#Create dataframe to hold the values
top_spenders = pd.DataFrame({"Purchase Count": count_purchase_sn,
                           "Average Purchase Price": average_purchase_sn.map("$ {:,.2f}".format),
                          "Total Purchase Value": total_purchase_sn.map("$ {:,.2f}".format)})
#Rearrange the columns
top_spenders = top_spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]


#Sort the values by Total Purchase Value descending and call the first 5 rows
top_spenders.sort_values("Total Purchase Value", ascending=False).head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Sundaky74,2,$ 3.71,$ 7.41
Aidaira26,2,$ 2.56,$ 5.13
Eusty71,1,$ 4.81,$ 4.81
Chanirra64,1,$ 4.78,$ 4.78
Alarap40,1,$ 4.71,$ 4.71


In [110]:
#Most Popular Items

#Identify the 5 most popular items by purchase count, then list (in a table):

#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

#Item Name
grouped_item = purchase.groupby(["Item ID","Item Name","Price"])

#Purchase Count
count_purchase_item = grouped_item["Purchase ID"].count()

#Create a dataframe to hold all values
items = pd.DataFrame({"Purchase Count": count_purchase_item})
items = items.reset_index()

#Sort top 5 items by Purchase Count
popular_items = items.sort_values("Purchase Count", ascending=False).head(5)

#Add a new column called Total Purchase Value
popular_items["Total Purchase Value"] = popular_items["Price"] * popular_items["Purchase Count"]

#Add $ signs after it has been sorted
popular_items["Price"] = popular_items["Price"].map(("$ {:,.2f}".format))
popular_items["Total Purchase Value"] = popular_items["Total Purchase Value"].map(("$ {:,.2f}".format))

#Rearrange columns
popular_items = popular_items[["Item ID", "Item Name", "Purchase Count", "Price", "Total Purchase Value"]]
popular_items



,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
31,94,Mourning Blade,3,$ 3.64,$ 10.92
28,90,Betrayer,2,$ 4.12,$ 8.24
38,111,Misery's End,2,$ 1.79,$ 3.58
21,64,Fusion Pummel,2,$ 2.42,$ 4.84
50,154,Feral Katana,2,$ 4.11,$ 8.22


In [117]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):

#Add new column with Total Purchase Value
items["Total Purchase Value"] = items["Price"] * items["Purchase Count"]

# #Sort top 5 items by Purchase Count
profitable_items = items.sort_values("Total Purchase Value", ascending=False).head(5)

# #Add $ signs after it has been sorted
profitable_items["Price"] = profitable_items["Price"].map(("$ {:,.2f}".format))
profitable_items["Total Purchase Value"] = profitable_items["Total Purchase Value"].map(("$ {:,.2f}".format))


# #Rearrange columns
profitable_items = profitable_items[["Item ID", "Item Name", "Purchase Count", "Price", "Total Purchase Value"]]
profitable_items

,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
31,94,Mourning Blade,3,$ 3.64,$ 10.92
39,117,"Heartstriker, Legacy of the Light",2,$ 4.71,$ 9.42
30,93,Apocalyptic Battlescythe,2,$ 4.49,$ 8.98
28,90,Betrayer,2,$ 4.12,$ 8.24
50,154,Feral Katana,2,$ 4.11,$ 8.22
